## Optimizing Dataframes memory
    In this project we are going to work with financial lending data from "Lending Club", a marketplace for personal loans that matches borrowers with investors. Our goal here is to optimize the dataframe and the processing in chunks.

In [1]:
import pandas as pd
pd.options.display.max_columns = 99    #We set the max as 99, so that we could see all the columns of the file

In [2]:
#Let's print the first 5 lines from loans_2007.csv to look for any data quality issues

file_head=pd.read_csv("loans_2007.csv", nrows=5)  #We use nrows since we don't want to read the entire file as it takes time

print(file_head)

        id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  1077501  1296599.0     5000.0       5000.0           4975.0   36 months   
1  1077430  1314167.0     2500.0       2500.0           2500.0   60 months   
2  1077175  1313524.0     2400.0       2400.0           2400.0   36 months   
3  1076863  1277178.0    10000.0      10000.0          10000.0   36 months   
4  1075358  1311748.0     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade                 emp_title emp_length  \
0   10.65%       162.87     B        B2                       NaN  10+ years   
1   15.27%        59.83     C        C4                     Ryder   < 1 year   
2   15.96%        84.33     C        C5                       NaN  10+ years   
3   13.49%       339.31     C        C1       AIR RESOURCES BOARD  10+ years   
4   12.69%        67.79     B        B5  University Medical Group     1 year   

  home_ownership  annual_inc verification_status  

In [3]:
#Calculating the chunk amount to keep the chunk's memory between 5 MB.

#First let's try with 1000 rows

first_1000 = pd.read_csv("loans_2007.csv", nrows=1000)

print(first_1000.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          1000 non-null   int64  
 1   member_id                   1000 non-null   float64
 2   loan_amnt                   1000 non-null   float64
 3   funded_amnt                 1000 non-null   float64
 4   funded_amnt_inv             1000 non-null   float64
 5   term                        1000 non-null   object 
 6   int_rate                    1000 non-null   object 
 7   installment                 1000 non-null   float64
 8   grade                       1000 non-null   object 
 9   sub_grade                   1000 non-null   object 
 10  emp_title                   949 non-null    object 
 11  emp_length                  983 non-null    object 
 12  home_ownership              1000 non-null   object 
 13  annual_inc                  1000 n

Having 1000 rows, shows memory usage of 1.5 MB. Therefore we can try to check for 3000 rows, to be between the 5 MB target.

In [4]:
first_3000 = pd.read_csv("loans_2007.csv", nrows=3000)

print(first_3000.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          3000 non-null   int64  
 1   member_id                   3000 non-null   float64
 2   loan_amnt                   3000 non-null   float64
 3   funded_amnt                 3000 non-null   float64
 4   funded_amnt_inv             3000 non-null   float64
 5   term                        3000 non-null   object 
 6   int_rate                    3000 non-null   object 
 7   installment                 3000 non-null   float64
 8   grade                       3000 non-null   object 
 9   sub_grade                   3000 non-null   object 
 10  emp_title                   2829 non-null   object 
 11  emp_length                  2917 non-null   object 
 12  home_ownership              3000 non-null   object 
 13  annual_inc                  3000 

As we expected, having 3000 rows shows only 4.6 MB which makes it as the perfect chunk size. 
Now, Let's check if the same memory usage consists across all other chunks and also count the total memory, no of rows the file has.

In [5]:
chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000)

total_row = 0

for chunk in chunk_iter:
    total_row+=len(chunk)
    print(chunk.memory_usage(deep=True).sum()/2**20)
    
print("\n", total_row)


4.580394744873047
4.576141357421875
4.577898979187012
4.579251289367676
4.575444221496582
4.577326774597168
4.575918197631836
4.578287124633789
4.576413154602051
4.57646369934082
4.589176177978516
4.588043212890625
4.594850540161133
4.828314781188965
0.868586540222168

 42538


The file loans_2007.csv has 42538 rows and 65.24 MB in total. We have also verified if the chunk's memory exceed more than 5 MB.

## How many column have a numeric type? 

Let's find out how many columns have numeric data type

In [6]:
chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000)

numeric_col=[]
string_col=[]

for chunk in chunk_iter:
    numeric_chunk=chunk.select_dtypes(include=['int64','float64'])   #Will output the complete dataframe that has only numeric dtype
    string_chunk=chunk.select_dtypes(include=['object'])       #Will output the complete dataframe that has only object dtype
    
    numeric_col.append(numeric_chunk.shape[1])    #this will append the no of col of each chunk into numeric_col
    string_col.append(string_chunk.shape[1])      #this will append the no of col of each chunk into string_col

#now the we have the list of numeric_col numbers, let's sum the numbers to get the total.
print(numeric_col)
print(string_col)


[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


We can see that, during the last 2 chunks of both numeric and object column, there is a discripency in the numbers.

In [7]:
#Let's check the missmatching chunk's columns

chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000)

obj_col=[]

for chunk in chunk_iter:
    obj_chunk=chunk.select_dtypes(include=['object']).columns.tolist()  #From the object dataframe, we took only the column names and made them into a list
    
    if len(obj_col) > 0:
        if obj_col != obj_chunk:      #checking if there is any column missmatch 
            print("\n","all_colums:",obj_col)
            print("\n","chunk_object_columns:",obj_chunk)
    else:
        obj_col=obj_chunk



 all_colums: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']

 chunk_object_columns: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']

 all_colums: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']

 chun

We can confirm here that, in the last 2 chunks the column 'id' is being shown which is supposed to be numeric column. So 'id' column has both numeric and object data type. Since id column won't be that useful for analysis we can ignore this column

## How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?

In [8]:
chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000)

col_val_counts={}

for chunk in chunk_iter:
    obj_columns=chunk.select_dtypes(include=['object']).columns.tolist()  #We took the names of a columns into a list
    for column in obj_columns:                                            #we used those column names to loop over dataframe  
        val_count=chunk[column].value_counts()                            
        if column in col_val_counts:
            col_val_counts[column].append(val_count)
        else:
            col_val_counts[column]=[val_count]                         #We set the value as list. so that we can append the key(columns) data of other chunks

unique_val_col={}

less_50_percentage_unique={}

less_50_unique_values={}          

for col in col_val_counts:
    u_concat=pd.concat(col_val_counts[col])             #The unique_val_counts dic has multiple chunks data. So we concat them into single series
    u_group=u_concat.groupby(u_concat.index).sum()         #When we concat, we might get duplicates, therefore we group them and sum their value
    unique_val_col[col]=len(u_group)
    
    if len(u_group) < 50:
        less_50_unique_values[col]=len(u_group)
    
    total_val=u_concat.sum()
    unique_percentage=(len(u_group)/total_val)*100
    
    if unique_percentage < 50:
        less_50_percentage_unique[col]=(len(u_group))
        

print("unique_values_in_each_column:",unique_val_col)

print("\n","Columns with less than 50% unique values:",less_50_percentage_unique)

print("\n", "Columns with less than 50 unique values:", less_50_unique_values)

unique_values_in_each_column: {'term': 2, 'int_rate': 394, 'grade': 7, 'sub_grade': 35, 'emp_title': 30658, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'issue_d': 55, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'title': 21264, 'zip_code': 837, 'addr_state': 50, 'earliest_cr_line': 530, 'revol_util': 1119, 'initial_list_status': 1, 'last_pymnt_d': 103, 'last_credit_pull_d': 108, 'application_type': 1, 'id': 3538}

 Columns with less than 50% unique values: {'term': 2, 'int_rate': 394, 'grade': 7, 'sub_grade': 35, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'issue_d': 55, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'zip_code': 837, 'addr_state': 50, 'earliest_cr_line': 530, 'revol_util': 1119, 'initial_list_status': 1, 'last_pymnt_d': 103, 'last_credit_pull_d': 108, 'application_type': 1}

 Columns with less than 50 unique values: {'term': 2, 'grade': 7, 'sub_grade': 35, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3,


Since some columns in "less_50_percentage_unique" have high cardinality, we chose to focus on columns with "less_50_unique_values" as they have very low cardinality.

## Which float columns have no missing values and could be candidates for conversion to the integer type?

In [9]:
chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000)

missing_counts=[]

for chunk in chunk_iter:
    float_chunk=chunk.select_dtypes(include=["float"])
    missing_counts.append(float_chunk.apply(pd.isnull).sum())    #apply.(pd.isnull) will Returns a DataFrame of the same shape as float_chunk, where each cell is True if the value is NaN and False otherwise. .sum() makes the column name as index and show the summed value
    
combined_missing=pd.concat(missing_counts)
combined_missing.groupby(combined_missing.index).sum().sort_values()
    




collection_recovery_fee          3
dti                              3
loan_amnt                        3
member_id                        3
last_pymnt_amnt                  3
installment                      3
funded_amnt_inv                  3
funded_amnt                      3
total_pymnt                      3
total_pymnt_inv                  3
total_rec_int                    3
revol_bal                        3
recoveries                       3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_prncp                  3
total_rec_late_fee               3
annual_inc                       7
open_acc                        32
delinq_amnt                     32
delinq_2yrs                     32
acc_now_delinq                  32
inq_last_6mths                  32
pub_rec                         32
total_acc                       32
tax_liens                      108
collections_12_mths_ex_med     148
chargeoff_within_12_

It seems there are no columns without missing values. However there are many columns with just 3 missing values which can be used for further optimization.

## Calculate the total memory usage across all chunks

In [10]:
chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000)

initial_memory=0

for chunk in chunk_iter:
    memory=chunk.memory_usage(deep=True).sum()/(1024*1024)
    initial_memory+=memory
    
print(initial_memory)  #Total memory that need to process the complete dataset


65.24251079559326


# Optimizing String Columns & Calculate the total memory footprint and compare it with the previous one.

Optimize the object columns by changing the data type of columns that has low cardinality into 'category' possible columns into numeric data type by cleaning them, changing the data columns into date type


In [11]:
print(less_50_unique_values) #Since we already found the less_50_unique_values we can use it to change into category & numeric


{'term': 2, 'grade': 7, 'sub_grade': 35, 'emp_length': 11, 'home_ownership': 5, 'verification_status': 3, 'loan_status': 9, 'pymnt_plan': 2, 'purpose': 14, 'initial_list_status': 1, 'application_type': 1}


In [12]:
#Based on our observation the columns 'term', 'int_rate', 'revol_util' are the potential columns that can be changed into numeric by doing some cleaning.

numeric_col=['term', 'int_rate', 'revol_util'] #potential numeric columns. These columns can be changed into numeric by cleaning them

category_dtype = {
    'grade': 'category', 
    'sub_grade': 'category',
    'emp_length': 'category',
    'home_ownership': 'category',
    'verification_status': 'category',
    'loan_status': 'category',
    'pymnt_plan': 'category',
    'purpose': 'category',
    'initial_list_status': 'category',
    'application_type': 'category'
}                                     #potential columns for category dtype which we will use while reading the chunks

date_columns=['last_pymnt_d', 'last_credit_pull_d', 'issue_d'] #potential columns for date dtype

In [13]:
chunk_iter=pd.read_csv("loans_2007.csv", chunksize=3000, dtype=category_dtype)


date_columns = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
date_format = "%b-%Y"


processed_chunks=[]
memory_aft_object_optimization=0

for chunk in chunk_iter:
    if 'term' in chunk.columns:
        chunk['term']=chunk['term'].str.rstrip(' months').astype('float')
    if 'int_rate' in chunk.columns:
        chunk['int_rate']=chunk['int_rate'].str.rstrip('%').astype('float')
    if 'revol_util' in chunk.columns:
        chunk['revol_util']=chunk['revol_util'].str.rstrip('%').astype('float')
    for col in date_columns:
        if col in chunk.columns:
            chunk[col] = pd.to_datetime(chunk[col], format=date_format)
    
    processed_chunks.append(chunk)
    
    memory=chunk.memory_usage(deep=True).sum()/(1024*1024)
    memory_aft_object_optimization+=memory

combined=pd.concat(processed_chunks)    #We are concating the output of all chunks together


print(" Memory before object dtype optimization:", initial_memory)
print("\n","Memory after object dtype optimization",memory_aft_object_optimization)

 Memory before object dtype optimization: 65.24251079559326

 Memory after object dtype optimization 23.905481338500977


# Optimizing the numeric columns and calculate the memory

In [14]:
import pandas as pd

chunk_iter = pd.read_csv("loans_2007.csv", chunksize=3000, dtype=category_dtype)

date_columns = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"]
date_format = "%b-%Y"

memory_aft_optimization=0
missing_counts = []  #Columns with missing values
float_dtypes = {}  #The exact dtypes of of the float columns

for chunk in chunk_iter:
    if 'term' in chunk.columns:
        chunk['term']=chunk['term'].str.rstrip(' months').astype('float')
    if 'int_rate' in chunk.columns:
        chunk['int_rate']=chunk['int_rate'].str.rstrip('%').astype('float')
    if 'revol_util' in chunk.columns:
        chunk['revol_util']=chunk['revol_util'].str.rstrip('%').astype('float')
    for col in date_columns:
        if col in chunk.columns:
            chunk[col] = pd.to_datetime(chunk[col], format=date_format)
       
    float_chunk = chunk.select_dtypes(include=["float"])   #Filtered columns with float type
        
    missing_counts.append(float_chunk.isnull().sum())     
    
    for col in float_chunk.columns:
        if col not in float_dtypes:
            float_dtypes[col] = float_chunk[col].dtype
        if float_chunk[col].dtype=='float64':    #We check the the dtype=='float64' and also if there is any missing values
            chunk[col] = pd.to_numeric(chunk[col], downcast='float')
    
    memory=chunk.memory_usage(deep=True).sum()/(1024*1024)
    memory_aft_optimization+=memory   

combined_missing = pd.concat(missing_counts)   #combining each chunk's results together

missing_values_summary = combined_missing.groupby(combined_missing.index).sum().sort_values()

No_missing_float_cols = missing_values_summary[missing_values_summary == 0 ]  #Filitering columns with only missing values to identify what columns we can change to int dtype
 
        
print("Columns and their missing values count:", "\n",missing_values_summary)

print("\n", "Float columns with no missing values:",No_missing_float_cols)      #let's checking columns missing value counts

print("\n", float_dtypes)               #Checking float columns actual datatype

print("\n", "Memory before numeric dtype optimization:", initial_memory)
print("\n","Memory after numeric dtype optimization:", memory_aft_optimization)
    



Columns and their missing values count: 
 collection_recovery_fee          3
dti                              3
last_pymnt_amnt                  3
loan_amnt                        3
int_rate                         3
installment                      3
funded_amnt_inv                  3
funded_amnt                      3
member_id                        3
total_rec_int                    3
total_pymnt_inv                  3
recoveries                       3
revol_bal                        3
policy_code                      3
out_prncp_inv                    3
out_prncp                        3
total_rec_late_fee               3
total_pymnt                      3
term                             3
total_rec_prncp                  3
annual_inc                       7
delinq_amnt                     32
inq_last_6mths                  32
acc_now_delinq                  32
total_acc                       32
pub_rec                         32
open_acc                        32
delinq_2yrs  

Observation:
1. We found no potential float columns that can be converted into integer since there were no columns with zero missing value
2. All the float columns were in float64 and many of them had missing values since we don't need much precision here we downcast them to optimize further
3. After downcasting and full optimization we were able to reduce the size down to 19 MB.

# Conculsion

By using chunk processing, optimizing data types (especially strings to categories and downcasting floats), parsing dates, and handling missing values, a substantial reduction in memory usage can be achieved for the "loans_2007.csv" dataset. This makes it more efficient to process the data, especially when dealing with large datasets.